In [1]:
import math
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import geopy.distance as dist

dir_string = "../data/20210716-19-scooterdata"
directory = os.fsencode(dir_string)
data = pd.DataFrame()
    
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    print('reading ',filename)
    if filename.endswith(".json"):
        newdata = pd.read_json(os.path.join(dir_string, filename))
        for i in range(newdata.shape[0]):
            if 'data' in newdata.alldata[i]:
                df = pd.DataFrame(newdata.alldata[i]['data']['bikes'])
                df.insert(0,'timestamp', newdata.alldata[i]['last_updated'])
                df.insert(0,'company',newdata.alldata[i]['company'])
                data = data.append(df, ignore_index=True)
                
data.to_csv(dir_string + '.csv', index=False)

reading  20210716_1352_scooter.json
reading  20210717_1630AM_scooter.json
reading  20210717_840AM_scooter.json
reading  20210718_705AM_scooter.json
reading  20210719_900AM_scooter.json


In [12]:
comp_ = ('link','wheels')
trip_count = np.zeros(len(comp_))
trip_time = np.zeros(len(comp_))
trips = pd.DataFrame()

for comp in comp_:
    df_comp = data[(data['company']==comp)&(data['is_disabled']==0)]
    ids=df_comp['bike_id'].unique()
    print(f'{len(ids)} unique {comp} ids')
    n = 0
    for scooter_id in ids:
        coords = df_comp[(df_comp['bike_id']==scooter_id)][['lat','lon','timestamp']].to_numpy()
        state = 'stopped'
        for i in range(coords.shape[0]-1):
            if dist.geodesic((coords[i,0],coords[i,1]),(coords[i+1,0],coords[i+1,1])).km>=0.1:
                if state != 'moving':
                    time_start = coords[i,2]
                    lat_start = coords[i,0]
                    lon_start = coords[i,1]
                state = 'moving'
            else:
                if state == 'moving':
                    state = 'end trip'
                    trips = trips.append({'bike_id':scooter_id, 'company':comp,
                                         'time_start':time_start, 'time_end':coords[i,2],
                                         'lat_start':lat_start, 'lon_start':lon_start,
                                         'lat_end':coords[i,0], 'lon_end':coords[i,1]},
                                         ignore_index=True)                    
                else:
                    state = 'stopped'
        n += 1
        if n%1000 == 0:
            print(f'{n} {comp} ids processed, {len(trips)} trips detected')

10891 unique link ids
1000 link ids processed, 856 trips detected
2000 link ids processed, 1815 trips detected
3000 link ids processed, 2871 trips detected
4000 link ids processed, 4046 trips detected
5000 link ids processed, 6032 trips detected
6000 link ids processed, 7739 trips detected
7000 link ids processed, 8421 trips detected
8000 link ids processed, 9277 trips detected
9000 link ids processed, 9909 trips detected
10000 link ids processed, 10662 trips detected
852 unique wheels ids


In [17]:
trips.to_csv('../data/20210716-19-trips.csv', index=False)